In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import training as tr
import emnet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

## Write events to a DataFrame

In [ ]:
datfile = "/home/jrenner/local/data/electronsim/pixelated_tracks_thinned_4um_back_10M_300keV.txt"

In [ ]:
def read_electron_data(fname, nevts=1000):
    
    evt = -1
    xinc = 0.0
    yinc = 0.0
    front = True
    energy = 0.0
    
    # Open the file and read the specified number of events.
    l_evt, l_xinc, l_yinc, l_front, l_energy, l_row, l_col, l_counts = [], [], [], [], [], [], [], []
    evts_read = 0
    with open(fname) as f:
        
        # Iterate through all lines.
        for line in f:
            
            # Stop reading if we've read the specified number of events.
            if(evts_read > nevts):
                break
            
            # Get each number in the line, separated by spaces.
            vals = line.rstrip().split(" ")
            
            # Start a new event.
            if(vals[0] == "EV"):
                evt    = vals[1]
                xinc   = vals[2]
                yinc   = vals[3]
                front  = (vals[4] == 1)
                energy = vals[5]
                evts_read += 1
                
            # Add a row for the current event.
            else:
                l_evt.append(int(evt))
                l_xinc.append(float(xinc))
                l_yinc.append(float(yinc))
                l_front.append(front)
                l_energy.append(float(energy))
                l_row.append(int(vals[0]))
                l_col.append(int(vals[1]))
                l_counts.append(int(vals[2]))
    
    # Construct the DataFrame.
    evt_dict = {'event': l_evt, 'xinc': l_xinc, 'yinc': l_yinc, 'front': l_front, 
                'energy': l_energy, 'row': l_row, 'col': l_col, 'counts': l_counts}
    df = pd.DataFrame.from_dict(evt_dict)
    
    return df

In [ ]:
df = read_electron_data(datfile,nevts=100005)

In [ ]:
len(df.event.unique())

In [ ]:
df.to_pickle("EM_4um_back_10M_300keV.pkl")

### Examine individual events

In [ ]:
evt_plt = 97
evt_arr = np.zeros([101,101])
df_evt = df[df.event == evt_plt]
for row,col,counts in zip(df_evt['row'].values,df_evt['col'].values,df_evt['counts'].values):
    evt_arr[row,col] += counts
plt.imshow(np.log(0.1 + evt_arr))
plt.colorbar()
plt.title("Event {}; max at {}".format(evt_plt,np.unravel_index(evt_arr.argmax(),evt_arr.shape)))

In [ ]:
max(df.xinc.values)

## Evaluate a dataset

In [ ]:
Nevts = 1000

In [ ]:
noise_arr = np.arange(0.,100.,50.)
r_mean, r_sigma = [], []
for noise in noise_arr:
    print("Running for noise",noise)
    dset = tr.EMDataset("EM_4um_back_10M_300keV.pkl",0,noise)
    
    shifts_x, shifts_y, shifts_r = [], [], []
    for evt in range(Nevts):
        evt_arr,evt_lbl = dset[evt]
        xs,ys = evt_lbl[0],evt_lbl[1]
        shifts_x.append(xs)
        shifts_y.append(ys)
        shifts_r.append((xs**2 + ys**2)**0.5)
    
    shifts_r = np.array(shifts_r)
    r_mean.append(np.mean(shifts_r))
    r_sigma.append(np.std(shifts_r))

In [ ]:
plt.errorbar(noise_arr,r_mean,yerr=np.array(r_sigma)/Nevts**0.5)
plt.xlabel("$\sigma$ noise (electrons)")
plt.ylabel("r-error (mm)")

### Read a dataset

In [ ]:
dset = tr.EMDataset("EM_4um_back_10M_300keV.pkl",noise_mean=0,noise_sigma=20,add_shift=10,add_noise=True)

In [ ]:
evt_plt = 8
evt_item = dset[evt_plt]
evt_arr = evt_item[0]
evt_lbl = evt_item[1]
evt_err_ind = evt_item[2]
plt.imshow(evt_arr)
plt.colorbar()
plt.title("Event {}; shift {}; index {}".format(evt_plt,evt_lbl,evt_err_ind))

In [ ]:
shifts_x, shifts_y, shifts_r = [], [], []
for evt in range(1000):
    evt_arr,evt_lbl = dset[evt]
    xs,ys = evt_lbl[0],evt_lbl[1]
    shifts_x.append(xs)
    shifts_y.append(ys)
    shifts_r.append((xs**2 + ys**2)**0.5)

In [ ]:
plt.hist(shifts_r,bins=100)
plt.yscale('log')

#### For debugging the 3x3 sum operations

In [ ]:
aa = np.ones([6,6])
aa[0,2] = 4
aa[1,2] = 2
aa[3,2] = 8
aa[4,2] = -2
aa[3,1] = 5
aa[5,0] = 10
aa

In [ ]:
max_init   = np.unravel_index(aa.argmax(),aa.shape)
nbsum_init = tr.sum_neighbors(aa,max_init,remove=True)
print("Max at",max_init,"and neighbor sum",nbsum_init)
aa

## Train a network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import training as tr
import emnet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [6]:
modeldir = '/home/jrenner/local/jerenner/emsim/models'
lrate       = 1e-4   # Learning rate to use in the training.
load_model  = True   # Load an existing model
tr.augment  = False  # Enable/disable data augmentation
epoch_start = 135      # Number of initial epoch
epoch_end   = 2000    # Number of final epoch
model_load_checkpoint = "{}/model_80cells_noise_100k_118.pt".format(modeldir)

# Create the datasets.
dataset_all   = tr.EMDataset("EM_4um_back_10M_300keV.pkl",noise_mean=0,noise_sigma=20,add_noise=True,add_shift=10)
dataset_train = tr.EMDataset("EM_4um_back_10M_300keV.pkl",noise_mean=0,noise_sigma=20,add_noise=True,nstart=0,nend=-20000,add_shift=10)
dataset_val   = tr.EMDataset("EM_4um_back_10M_300keV.pkl",noise_mean=0,noise_sigma=20,add_noise=True,nstart=-20000,add_shift=10)

# Create the loaders.
train_loader = DataLoader(dataset_train, batch_size=1000, shuffle=True, collate_fn=tr.my_collate, num_workers=8)
val_loader = DataLoader(dataset_val, batch_size=1000, shuffle=True, collate_fn=tr.my_collate, num_workers=8)
#val_loader = DataLoader(dataset_val, batch_size=15, shuffle=True, collate_fn=tr.my_collate, num_workers=4)
#test_loader = DataLoader(dataset_test, batch_size=15, shuffle=True, collate_fn=tr.my_collate, num_workers=4)

# Define the model.
#model = emnet.FCNet()
model = emnet.basicCNN()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lrate, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.01, amsgrad=True)

# Load the model from file.
if(load_model):
    model.load_state_dict(torch.load(model_load_checkpoint))
    #model.load_state_dict(torch.load(model_load_checkpoint,map_location=torch.device('cpu')))
    model.eval()

Created dataset for events from 0 to 100000
Created dataset for events from 0 to -20000
Created dataset for events from -20000 to 20000


In [7]:
# Run the training.
#print("Training with weights",sort_clsweights)
for epoch in range(epoch_start,epoch_end):
    print("Epoch: ",epoch)
    model.train()
    tr.train(model, epoch, train_loader, optimizer)
    with torch.no_grad():
        model.eval()
        tr.val(model, epoch, val_loader)
#     if(epoch % 50 == 0):
#         torch.save(model.state_dict(), "{}/model_init_{}.pt".format(modeldir,epoch))

Epoch:  119
Train Epoch: 119 [0/80000 (0%)]	Loss: 6.332152	 score_max: 12.998805	 score_min: -3.188294; Accuracy 0.021
Train Epoch: 119 [1000/80000 (1%)]	Loss: 6.310079	 score_max: 13.252916	 score_min: -2.710984; Accuracy 0.038
Train Epoch: 119 [2000/80000 (2%)]	Loss: 6.262750	 score_max: 15.926132	 score_min: -2.788737; Accuracy 0.036
Train Epoch: 119 [3000/80000 (4%)]	Loss: 6.271316	 score_max: 11.040543	 score_min: -2.296937; Accuracy 0.031
Train Epoch: 119 [4000/80000 (5%)]	Loss: 6.354444	 score_max: 12.042006	 score_min: -2.175408; Accuracy 0.027
Train Epoch: 119 [5000/80000 (6%)]	Loss: 6.342536	 score_max: 12.195705	 score_min: -2.750943; Accuracy 0.023
Train Epoch: 119 [6000/80000 (8%)]	Loss: 6.291351	 score_max: 10.932170	 score_min: -2.504694; Accuracy 0.022
Train Epoch: 119 [7000/80000 (9%)]	Loss: 6.219892	 score_max: 12.116998	 score_min: -2.401964; Accuracy 0.039
Train Epoch: 119 [8000/80000 (10%)]	Loss: 6.247897	 score_max: 12.496360	 score_min: -2.489066; Accuracy 0.038


Train Epoch: 119 [76000/80000 (95%)]	Loss: 6.319593	 score_max: 10.613869	 score_min: -2.432445; Accuracy 0.027
Train Epoch: 119 [77000/80000 (96%)]	Loss: 6.284318	 score_max: 14.417966	 score_min: -2.949447; Accuracy 0.031
Train Epoch: 119 [78000/80000 (98%)]	Loss: 6.298709	 score_max: 11.222418	 score_min: -2.169338; Accuracy 0.025
Train Epoch: 119 [79000/80000 (99%)]	Loss: 6.280334	 score_max: 10.431209	 score_min: -2.228894; Accuracy 0.026
---EPOCH AVG TRAIN LOSS: 6.311801266670227 ACCURACY: 0.028950001182965934
--Val Epoch: 119 [0/20000 (0%)]	Loss: 6.166696	 score_max: 7.254490	 score_min: -1.306777; Accuracy 0.070
--Val Epoch: 119 [1000/20000 (5%)]	Loss: 6.139235	 score_max: 6.501732	 score_min: -1.226601; Accuracy 0.072
--Val Epoch: 119 [2000/20000 (10%)]	Loss: 6.112737	 score_max: 6.265472	 score_min: -1.221668; Accuracy 0.070
--Val Epoch: 119 [3000/20000 (15%)]	Loss: 6.138201	 score_max: 6.362390	 score_min: -1.315677; Accuracy 0.072
--Val Epoch: 119 [4000/20000 (20%)]	Loss: 6

Train Epoch: 120 [50000/80000 (62%)]	Loss: 6.285986	 score_max: 14.891874	 score_min: -2.470846; Accuracy 0.036
Train Epoch: 120 [51000/80000 (64%)]	Loss: 6.267819	 score_max: 11.225553	 score_min: -2.033800; Accuracy 0.028
Train Epoch: 120 [52000/80000 (65%)]	Loss: 6.324926	 score_max: 16.114780	 score_min: -3.002864; Accuracy 0.025
Train Epoch: 120 [53000/80000 (66%)]	Loss: 6.215802	 score_max: 18.752342	 score_min: -3.043700; Accuracy 0.036
Train Epoch: 120 [54000/80000 (68%)]	Loss: 6.324399	 score_max: 12.218266	 score_min: -2.209586; Accuracy 0.030
Train Epoch: 120 [55000/80000 (69%)]	Loss: 6.243432	 score_max: 12.747385	 score_min: -2.197395; Accuracy 0.035
Train Epoch: 120 [56000/80000 (70%)]	Loss: 6.336899	 score_max: 11.909145	 score_min: -2.112841; Accuracy 0.021
Train Epoch: 120 [57000/80000 (71%)]	Loss: 6.329468	 score_max: 11.659468	 score_min: -2.256464; Accuracy 0.026
Train Epoch: 120 [58000/80000 (72%)]	Loss: 6.298933	 score_max: 13.246428	 score_min: -2.722594; Accurac

Train Epoch: 121 [24000/80000 (30%)]	Loss: 6.238816	 score_max: 10.837552	 score_min: -2.134841; Accuracy 0.035
Train Epoch: 121 [25000/80000 (31%)]	Loss: 6.248693	 score_max: 13.134314	 score_min: -2.188661; Accuracy 0.030
Train Epoch: 121 [26000/80000 (32%)]	Loss: 6.305545	 score_max: 11.164948	 score_min: -2.243209; Accuracy 0.030
Train Epoch: 121 [27000/80000 (34%)]	Loss: 6.299708	 score_max: 10.606471	 score_min: -2.268591; Accuracy 0.030
Train Epoch: 121 [28000/80000 (35%)]	Loss: 6.346797	 score_max: 12.016502	 score_min: -2.231070; Accuracy 0.033
Train Epoch: 121 [29000/80000 (36%)]	Loss: 6.318181	 score_max: 11.192270	 score_min: -2.327445; Accuracy 0.029
Train Epoch: 121 [30000/80000 (38%)]	Loss: 6.233716	 score_max: 11.149525	 score_min: -2.138995; Accuracy 0.018
Train Epoch: 121 [31000/80000 (39%)]	Loss: 6.371395	 score_max: 11.238135	 score_min: -2.150406; Accuracy 0.032
Train Epoch: 121 [32000/80000 (40%)]	Loss: 6.336159	 score_max: 12.444864	 score_min: -2.369721; Accurac

Train Epoch: 122 [0/80000 (0%)]	Loss: 6.270154	 score_max: 13.529506	 score_min: -2.307710; Accuracy 0.033
Train Epoch: 122 [1000/80000 (1%)]	Loss: 6.253198	 score_max: 10.969093	 score_min: -2.620398; Accuracy 0.027
Train Epoch: 122 [2000/80000 (2%)]	Loss: 6.283020	 score_max: 11.202319	 score_min: -2.164992; Accuracy 0.039
Train Epoch: 122 [3000/80000 (4%)]	Loss: 6.304257	 score_max: 12.108388	 score_min: -2.499186; Accuracy 0.034
Train Epoch: 122 [4000/80000 (5%)]	Loss: 6.325381	 score_max: 12.254750	 score_min: -2.220885; Accuracy 0.028
Train Epoch: 122 [5000/80000 (6%)]	Loss: 6.326350	 score_max: 11.749969	 score_min: -2.594520; Accuracy 0.037
Train Epoch: 122 [6000/80000 (8%)]	Loss: 6.241752	 score_max: 10.116453	 score_min: -1.977229; Accuracy 0.032
Train Epoch: 122 [7000/80000 (9%)]	Loss: 6.187841	 score_max: 14.665723	 score_min: -2.571669; Accuracy 0.027
Train Epoch: 122 [8000/80000 (10%)]	Loss: 6.203986	 score_max: 12.141952	 score_min: -2.816704; Accuracy 0.031
Train Epoch:

...

--Val Epoch: 131 [9000/20000 (45%)]	Loss: 6.098392	 score_max: 6.222520	 score_min: -1.240763; Accuracy 0.084
--Val Epoch: 131 [10000/20000 (50%)]	Loss: 6.044093	 score_max: 12.833416	 score_min: -2.963508; Accuracy 0.083
--Val Epoch: 131 [11000/20000 (55%)]	Loss: 6.068141	 score_max: 6.488943	 score_min: -1.329219; Accuracy 0.095
--Val Epoch: 131 [12000/20000 (60%)]	Loss: 5.969976	 score_max: 6.735074	 score_min: -1.246469; Accuracy 0.087
--Val Epoch: 131 [13000/20000 (65%)]	Loss: 6.078535	 score_max: 6.340767	 score_min: -1.209364; Accuracy 0.088
--Val Epoch: 131 [14000/20000 (70%)]	Loss: 6.110623	 score_max: 6.834400	 score_min: -1.504093; Accuracy 0.091
--Val Epoch: 131 [15000/20000 (75%)]	Loss: 6.106627	 score_max: 5.949099	 score_min: -1.218798; Accuracy 0.068
--Val Epoch: 131 [16000/20000 (80%)]	Loss: 6.037360	 score_max: 6.315274	 score_min: -1.257711; Accuracy 0.095
--Val Epoch: 131 [17000/20000 (85%)]	Loss: 6.085230	 score_max: 6.306887	 score_min: -1.272559; Accuracy 0.090
-

Train Epoch: 132 [63000/80000 (79%)]	Loss: 6.184595	 score_max: 12.306549	 score_min: -2.113519; Accuracy 0.037
Train Epoch: 132 [64000/80000 (80%)]	Loss: 6.255662	 score_max: 14.036246	 score_min: -2.268444; Accuracy 0.032
Train Epoch: 132 [65000/80000 (81%)]	Loss: 6.197971	 score_max: 10.900501	 score_min: -2.436034; Accuracy 0.038
Train Epoch: 132 [66000/80000 (82%)]	Loss: 6.233894	 score_max: 12.746982	 score_min: -2.482766; Accuracy 0.036
Train Epoch: 132 [67000/80000 (84%)]	Loss: 6.254736	 score_max: 11.206634	 score_min: -2.379508; Accuracy 0.028
Train Epoch: 132 [68000/80000 (85%)]	Loss: 6.258584	 score_max: 11.467291	 score_min: -2.108988; Accuracy 0.034
Train Epoch: 132 [69000/80000 (86%)]	Loss: 6.215518	 score_max: 10.230888	 score_min: -2.100504; Accuracy 0.030
Train Epoch: 132 [70000/80000 (88%)]	Loss: 6.138576	 score_max: 13.452100	 score_min: -2.573895; Accuracy 0.028
Train Epoch: 132 [71000/80000 (89%)]	Loss: 6.107736	 score_max: 11.845304	 score_min: -2.092687; Accurac

Train Epoch: 133 [37000/80000 (46%)]	Loss: 6.159525	 score_max: 10.147020	 score_min: -2.177932; Accuracy 0.038
Train Epoch: 133 [38000/80000 (48%)]	Loss: 6.268421	 score_max: 12.869050	 score_min: -2.238458; Accuracy 0.024
Train Epoch: 133 [39000/80000 (49%)]	Loss: 6.220698	 score_max: 10.454144	 score_min: -1.942212; Accuracy 0.036
Train Epoch: 133 [40000/80000 (50%)]	Loss: 6.272142	 score_max: 11.785503	 score_min: -2.268214; Accuracy 0.030
Train Epoch: 133 [41000/80000 (51%)]	Loss: 6.258777	 score_max: 11.399404	 score_min: -2.062659; Accuracy 0.033
Train Epoch: 133 [42000/80000 (52%)]	Loss: 6.308174	 score_max: 14.742496	 score_min: -2.203923; Accuracy 0.030
Train Epoch: 133 [43000/80000 (54%)]	Loss: 6.271720	 score_max: 11.572718	 score_min: -2.044942; Accuracy 0.023
Train Epoch: 133 [44000/80000 (55%)]	Loss: 6.250991	 score_max: 10.717246	 score_min: -2.274182; Accuracy 0.026
Train Epoch: 133 [45000/80000 (56%)]	Loss: 6.281748	 score_max: 11.850967	 score_min: -2.409184; Accurac

Train Epoch: 134 [15000/80000 (19%)]	Loss: 6.179763	 score_max: 11.325722	 score_min: -2.239700; Accuracy 0.038
Train Epoch: 134 [16000/80000 (20%)]	Loss: 6.226953	 score_max: 10.615458	 score_min: -2.419001; Accuracy 0.036
Train Epoch: 134 [17000/80000 (21%)]	Loss: 6.124116	 score_max: 10.033920	 score_min: -2.451800; Accuracy 0.034
Train Epoch: 134 [18000/80000 (22%)]	Loss: 6.173594	 score_max: 12.343359	 score_min: -1.950393; Accuracy 0.024
Train Epoch: 134 [19000/80000 (24%)]	Loss: 6.155266	 score_max: 12.059790	 score_min: -2.410079; Accuracy 0.036
Train Epoch: 134 [20000/80000 (25%)]	Loss: 6.185688	 score_max: 12.417426	 score_min: -2.274621; Accuracy 0.033
Train Epoch: 134 [21000/80000 (26%)]	Loss: 6.213542	 score_max: 13.096833	 score_min: -2.126385; Accuracy 0.032
Train Epoch: 134 [22000/80000 (28%)]	Loss: 6.230761	 score_max: 10.481774	 score_min: -2.190267; Accuracy 0.036
Train Epoch: 134 [23000/80000 (29%)]	Loss: 6.252441	 score_max: 11.005272	 score_min: -2.010682; Accurac

KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), "{}/model_80cells_noise_100k_{}.pt".format(modeldir,epoch))

### Test the net

In [ ]:
dset = tr.EMDataset("EM_4um_back_10M_300keV.pkl",noise_mean=0,noise_sigma=20,add_noise=True,add_shift=10)
softmax = nn.Softmax(dim=1)
model.eval()

In [ ]:
evt_plt = 9801
evt_item = dset[evt_plt]
evt_arr = evt_item[0]
evt_lbl = evt_item[1]
evt_err_ind = evt_item[2]

SHIFTED_ERR_RANGE_MIN = tr.PIXEL_ERR_RANGE_MIN - dset.add_shift*tr.PIXEL_SIZE
SHIFTED_ERR_RANGE_MAX = tr.PIXEL_ERR_RANGE_MAX + dset.add_shift*tr.PIXEL_SIZE

xbin = int(tr.ERR_SIZE*(evt_lbl[0] - SHIFTED_ERR_RANGE_MIN)/(SHIFTED_ERR_RANGE_MAX - SHIFTED_ERR_RANGE_MIN))
xbin = max(xbin,0)
xbin = min(xbin,tr.ERR_SIZE-1)

ybin = int(tr.ERR_SIZE*(evt_lbl[1] - SHIFTED_ERR_RANGE_MIN)/(SHIFTED_ERR_RANGE_MAX - SHIFTED_ERR_RANGE_MIN))
ybin = max(ybin,0)
ybin = min(ybin,tr.ERR_SIZE-1)

print("Computed index:",(ybin*tr.ERR_SIZE) + xbin,"for max added shift:",dset.add_shift)

# Send through the model.
data = torch.tensor(evt_arr).float().unsqueeze(0).unsqueeze(1).cuda()
target = torch.tensor(np.array(evt_err_ind)).long().cuda()

output_score = model(data)
prob = np.array(softmax(output_score).cpu().detach().numpy()).reshape([tr.ERR_SIZE,tr.ERR_SIZE])

fig = plt.figure()
fig.set_figheight(6.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121)
plt.imshow(evt_arr)
plt.colorbar()
plt.title("Event {}; shift ({:.3e},{:.3e}); index {}".format(evt_plt,evt_lbl[0],evt_lbl[1],evt_err_ind))

ax2 = fig.add_subplot(122)
ax2.plot([xbin],[ybin],color='red',marker='o',markersize=10)
plt.imshow(prob)
plt.colorbar()
plt.title("Incidence point within central pixel")

In [ ]:
# Look at many events:
xpred_err, ypred_err = [], []
for evt_plt in np.arange(8000,9999):
    
    evt_item = dset[evt_plt]
    evt_arr = evt_item[0]
    evt_lbl = evt_item[1]
    evt_err_ind = evt_item[2]

    # Send through the model.
    data = torch.tensor(evt_arr).float().unsqueeze(0).unsqueeze(1).cuda()
    target = torch.tensor(np.array(evt_err_ind)).long().cuda()

    output_score = model(data)
    prob = np.argmax(np.array(softmax(output_score).cpu().detach().numpy()).reshape([10,10]))
    xpred = (prob % tr.ERR_SIZE)*0.005/tr.ERR_SIZE + tr.ERR_RANGE_MIN + 0.005/tr.ERR_SIZE/2
    ypred = (prob / tr.ERR_SIZE)*0.005/tr.ERR_SIZE + tr.ERR_RANGE_MIN + 0.005/tr.ERR_SIZE/2
    print("[Evt",evt_plt,"]: Index is",evt_err_ind,"with predicted",prob,"; x = {} (predicted {}), y = {} (predicted {})".format(evt_lbl[0],xpred,evt_lbl[1],ypred))
    
    xpred_err.append(xpred-evt_lbl[0])
    ypred_err.append(ypred-evt_lbl[1])
xpred_err = np.array(xpred_err)
ypred_err = np.array(ypred_err)

In [ ]:
fig = plt.figure()
fig.set_figheight(6.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(xpred_err)
plt.xlabel("error in x-prediction (mm)")
print(np.where(abs(xpred_err) > 0.001))

ax2 = fig.add_subplot(122)
plt.hist(ypred_err)
plt.xlabel("error in y-prediction (mm)")
